In [1]:
import findspark

findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, struct, lit, array

In [4]:
spark = SparkSession.builder.appName("test").master("local").getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/spark-3.1.2-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mickeyvu0811/.ivy2/cache
The jars for the packages stored in: /home/mickeyvu0811/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-19ae1cab-3aee-4187-9cbc-01b360792955;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
	found com.datastax.spark#spark-cassandra

Column<'struct(INSERT AS `operation`, NULL AS `oldValue` AS `partyAddress`)'>

In [8]:
df = spark.read \
.format("csv") \
.option("path", "../test_data/accounts/account_samples.csv") \
.option("header", True) \
.load()

In [9]:
df.show()

+----------+----------+----------+----------+--------------------+------------------+--------------+-----------+------------------+-----------+-------------+
| load_date|active_ind|account_id|source_sys|  account_start_date|     legal_title_1| legal_title_2|tax_id_type|            tax_id|branch_code|      country|
+----------+----------+----------+----------+--------------------+------------------+--------------+-----------+------------------+-----------+-------------+
|2022-08-02|         1|6982391060|       COH|2018-03-24T13:56:...|     Tiffany Riley|Matthew Davies|        EIN|ZLCK91795330413525|   ACXMGBA5|       Mexico|
|2022-08-02|         1|6982391061|       ADS|2018-07-19T11:24:...|   Garcia and Sons| Taylor Guzman|        SSP|CADU39916151090321|   SHJFGBML|United States|
|2022-08-02|         1|6982391067|       BDL|2018-08-29T17:18:...|        Acosta Inc|  David Walker|        SSP|UJLN20870916345792|   WZTEGBTG|       Canada|
|2022-08-02|         0|6982391063|       ADS|2019-02

In [14]:
df = spark.read \
.format("csv") \
.option("path", "../test_data/party_address/address_samples.csv") \
.option("header", True) \
.load()

In [15]:
df.show()

+----------+----------+--------------------+--------------------+-------------------+-----------+------------------+------------------+
| load_date|  party_id|      address_line_1|      address_line_2|               city|postal_code|country_of_address|address_start_date|
+----------+----------+--------------------+--------------------+-------------------+-----------+------------------+------------------+
|2022-08-02|9823462810|   45229 Drake Route|   13306 Corey Point|          Shanefort|      77163|            Canada|        2019-02-26|
|2022-08-02|9823462811|  361 Robinson Green|3511 Rebecca Mission|        North Tyler|      34118|            Canada|        2018-01-28|
|2022-08-02|9823462812|    039 Daniel Mount|8219 Hernandez Lo...|      Boltonborough|      71648|            Mexico|        2018-12-07|
|2022-08-02|9823462813|  05550 Nancy Rapids| 9471 Zachary Canyon|     East Davidport|      02504|     United States|        2019-04-02|
|2022-08-02|9823462814|   5227 Wagner Pines|189 

In [16]:
address = struct(
    col("address_line_1").alias("addressLine1"),
    col("address_line_2").alias("addressLine2"),
    col("city").alias("addressCity"),
    col("postal_code").alias("addressPostalCode"),
    col("country_of_address").alias("addressCountry"),
    col("address_start_date").alias("addressStartDate")
)

In [20]:
df.select("party_id", struct(lit("INSERT").alias("operation"), address.alias("newValue"), lit(None).alias("oldValue").alias("partyAddress"))).show(truncate=False)

+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|party_id  |struct(INSERT AS `operation`, struct(address_line_1 AS `addressLine1`, address_line_2 AS `addressLine2`, city AS `addressCity`, postal_code AS `addressPostalCode`, country_of_address AS `addressCountry`, address_start_date AS `addressStartDate`) AS `newValue`, NULL AS `oldValue` AS `partyAddress`)|
+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|9823462810|{INSERT, {45229 Drake Route, 13306 Corey Point, Shan